In [2]:
import mysql.connector 

In [3]:
from mysql.connector import (pooling, Error)

In [6]:
try:
    # Connection configuration
    connection_config_dict = {
        'host': 'localhost',
        'user': 'root',
        'database': 'little_lemon_db',
        'autocommit': True
    }

    # Connect to the database using mysql.connector
    connection = mysql.connector.connect(**connection_config_dict)

    # Create a connection pool
    pool = pooling.MySQLConnectionPool(pool_name="little_lemon_pool",
                                       pool_size=2,
                                       **connection_config_dict)

    print("The connection pool is created with the name:", pool.pool_name, "and the pool size is:", pool.pool_size)

    # Get a cursor object from the connection
    cursor = connection.cursor()

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version", db_Info)

        # Global connection timeout arguments
        global_connect_timeout = 'SET GLOBAL connect_timeout=180'
        global_wait_timeout = 'SET GLOBAL wait_timeout=180'
        global_interactive_timeout = 'SET GLOBAL interactive_timeout=180'

        cursor.execute(global_connect_timeout)
        cursor.execute(global_wait_timeout)
        cursor.execute(global_interactive_timeout)

        connection.commit()

        # Use the little_lemon_db database
        use_database_query = """USE little_lemon_db;"""
        cursor.execute(use_database_query)
        print("Using little lemon database")

        # Query to get the name and EmployeeID of the Little Lemon manager
        manager_query = """SELECT EmployeeID, Name FROM Employees WHERE Role = 'Manager';"""
        cursor.execute(manager_query)

        manager_data = cursor.fetchone()
        print()
        print(manager_data)

        # Query to get the name and role of the employee who receives the highest salary
        cursor.execute("SELECT Name, Role FROM Employees ORDER BY Annual_Salary DESC LIMIT 1;")
        highest_salary_employee_data = cursor.fetchone()

        if highest_salary_employee_data:
            highest_salary_employee_name, highest_salary_employee_role = highest_salary_employee_data
            print("Employee with Highest Salary:")
            print(f"Name: {highest_salary_employee_name}, Role: {highest_salary_employee_role}")
            print()
        else:
            print("No employee with the highest salary found.")

        # Query to count the number of guests booked between 18:00 and 20:00
        print("Booking from the booking slot between 18:00 and 20:00")
        cursor.execute("SELECT COUNT(*) FROM Bookings WHERE BookingSlot BETWEEN '18:00' AND '20:00'")
        num_guests_booked = cursor.fetchone()[0]
        print("Number of Guests Booked between 18:00 and 20:00:", num_guests_booked)
        print()

        # Query to get the full name and BookingID of guests waiting to be seated, sorted by BookingSlot
        
        cursor.execute("SELECT CONCAT(GuestFirstName, ' ', GuestLastName) AS FullName, BookingID FROM Bookings WHERE TableNo>0  ORDER BY BookingSlot ASC")
        waiting_guests = cursor.fetchall()
        print()
        print(waiting_guests)

        

except Error as e:
    # Handle generic MySQL errors
    print("MySQL Error:", e)

except Exception as ex:
    # Handle any other type of exception not caught by previous blocks
    print("Unknown Error:", ex)

finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")


The connection pool is created with the name: little_lemon_pool and the pool size is: 2
Connected to MySQL Server version 5.5.5-10.4.28-MariaDB
Using little lemon database

(1, 'Mario Gollini')
Employee with Highest Salary:
Name: Mario Gollini, Role: Manager

Booking from the booking slot between 18:00 and 20:00
Number of Guests Booked between 18:00 and 20:00: 7


[('Vanessa McCarthy', 3), ('Marcos Romero', 4), ('Anees Java', 7), ('Hiroki Yamane', 5), ('Anna Iversen', 1), ('Joakim Iversen', 2), ('Bald Vin', 8), ('Jay Kon', 9), ('Diana Pinto', 6)]
MySQL connection is closed
